In [1]:
# import pysam
# from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
from statsmodels.stats.multitest import multipletests

In [2]:
mhc_repeat_region=[[32589647, 32751796]]

In [3]:
dis_split_region={'ACB': {'chr3': [[98029388, 98331063]],
  'chr4': [[8996392, 9139446]],
  'chr5': [[34277945, 34574089]]},
 'ASW': {'chr1': [[248297800, 248584618]]},
 'BEB': {'chr13': [[55108214, 55447878]]},
 'CDX': {'chr7': [[64033450, 64433853]], 'chr15': [[24061424, 24264850]]},
 'CEU': {'chr17': [[45825257, 46352295]]},
 'CHB': {},
 'CHS': {},
 'CLM': {'chr3': [[95042564, 95345546],
   [98029388, 98261301],
   [163846034, 164208695]],
  'chr17': [[45825257, 46352295]]},
 'ESN': {'chr1': [[248297800, 248584618]], 'chr5': [[34277945, 34574089]]},
 'FIN': {},
 'GBR': {'chr1': [[227003796, 227338558]],
  'chr3': [[98029388, 98261301]],
  'chr17': [[45825257, 46352295]]},
 'GIH': {'chr3': [[98029388, 98331063]], 'chr14': [[19806333, 20078222]]},
 'GWD': {'chr1': [[248297800, 248584618]],
  'chr5': [[34277945, 34574089]],
  'chr7': [[11926418, 12230106]]},
 'IBS': {'chr1': [[227003796, 227338558]], 'chr17': [[45825257, 46352295]]},
 'ITU': {'chr17': [[52570659, 52892893]]},
 'JPT': {},
 'KHV': {},
 'LWK': {'chr4': [[8996392, 9139446]]},
 'MSL': {'chr5': [[34231981, 34574089]]},
 'MXL': {'chr1': [[227003796, 227338558]], 'chr11': [[124162560, 124482002]]},
 'PEL': {'chr3': [[98029388, 98331063]], 'chr11': [[23181031, 23574546]]},
 'PJL': {'chr3': [[163846034, 164208695]]},
 'PUR': {'chr3': [[98029388, 98331063]], 'chr17': [[45825257, 46352295]]},
 'STU': {'chr3': [[98029388, 98293778]], 'chr13': [[55108214, 55447878]]},
 'TSI': {'chr7': [[67111221, 67437767]], 'chr17': [[45825257, 46352295]]},
 'YRI': {'chr1': [[227003796, 227338558], [248297800, 248584618]],
  'chr3': [[166091993, 166408110]],
  'chr5': [[34277945, 34574089]]}}

In [4]:
ass_split_race={'ACB': {'chr2': [[92299419, 92523478]],
  'chr3': [[89474731, 90288204]],
  'chr5': [[62144306, 62605777]],
  'chr14': [[105863438, 106259905]],
  'chr16': [[36332499, 46403568]],
  'chr18': [[19764009, 20828994]],
  'chr22': [[22691667, 22898645]]},
  'All':{'chr2': [[108377615, 108667023]],
 'chr3': [[87292011, 87555666]],
 'chr9': [[88434984, 88700396]],
 'chr11': [[39680311, 39930646]],
 'chr12': [[20836055, 21097074], [34328866, 34516639]],
 'chr14': [[31627550, 31888106]]},
 'ASW': {'chr2': [[92299419, 92507872],
   [92633551, 92902449],
   [93680283, 94160185],
   [94892770, 95546698],
   [202715454, 203549066]],
  'chr3': [[93559360, 94116184]],
  'chr8': [[49383975, 49798135], [103467484, 103900956]],
  'chr10': [[72960046, 73409340]],
  'chr14': [[105859287, 106198900]],
  'chr16': [[36260059, 46728534]]},
 'BEB': {'chr15': [[77138940, 77566277]]},
 'CDX': {'chr8': [[70075248, 70483016]],
  'chr11': [[50311095, 50728899]],
  'chr14': [[66379055, 66778338]],
  'chr20': [[21834643, 22233777]]},
 'CEU': {'chr8': [[84442692, 84821607]],
  'chr12': [[34328866, 34590462]],
  'chr14': [[105917600, 106259905]],
  'chr22': [[22661982, 22898645]]},
 'CHB': {'chr6_partC': [[85501992, 86104633]],
  'chr11': [[47086853, 47921826]]},
 'CHS': {'chr10': [[63107761, 63493252]],
  'chr13': [[47866045, 48251299]],
  'chr15': [[76471437, 76909646]]},
 'CLM': {'chr19': [[42860739, 43046093]]},
 'ESN': {'chr12': [[72479117, 72899345]]},
 'FIN': {'chr3': [[75380511, 75596024]],
  'chr5': [[131238500, 132001329]],
  'chr17': [[39204905, 39592604], [45610677, 46138242]]},
 'GBR': {'chr19': [[22622704, 23047137]], 'chr22': [[41329774, 41808259]]},
 'GIH': {'chr3': [[157950988, 158348010]],
  'chr6_partC': [[85696088, 86044297]],
  'chr12': [[132935977, 133264749]],
  'chr19': [[42820338, 43026385]]},
 'GWD': {'chr2': [[213229719, 213686213]],
  'chr3': [[47078886, 47548303]],
  'chr10': [[38164827, 38614638]]},
 'IBS': {'chr3': [[75380511, 75596024]]},
 'ITU': {'chr1': [[189001539, 189336299]],
  'chr8': [[50652103, 51059321]],
  'chr10': [[72260319, 72661979]],
  'chr12': [[32954541, 33301513]]},
 'JPT': {'chr10': [[73001439, 73409340]], 'chr14': [[66187582, 66778338]]},
 'KHV': {'chr2': [[186057591, 186516143]],
  'chr4': [[151301085, 151752781]],
  'chr10': [[38073145, 38635036], [73001439, 73409340]],
  'chr17': [[58394956, 59118140]]},
 'LWK': {'chr2': [[93680283, 94160185]],
  'chr9': [[64439637, 65054326]],
  'chr10': [[102847077, 103283225]],
  'chr14': [[105863438, 106385732]],
  'chr16': [[36332499, 46403568]],
  'chr22': [[22691667, 22898645]]},
 'MSL': {'chr2': [[93680283, 94143754]],
  'chr5': [[46140397, 46434522]],
  'chr10': [[38073145, 38635036]],
  'chr14': [[105863438, 106259905]],
  'chr18': [[19764009, 20828994]],
  'chr19': [[27564615, 27844019]],
  'chr20': [[18364231, 18744031]],
  'chr22': [[22691667, 22898645]]},
 'MXL': {'chr3': [[94780047, 95118014]], 'chr11': [[49588569, 50709669]]},
 'PEL': {'chr4': [[150649663, 151256009]],
  'chr5': [[46140397, 46434522]],
  'chr11': [[48330435, 49133873]]},
 'PJL': {'chr6_partC': [[118358152, 118798801]],
  'chr7': [[57291163, 57656843]]},
 'PUR': {'chr11': [[49133908, 50818091], [54704427, 55220212]]},
 'STU': {'chr2': [[135176374, 135572114]],
  'chr3': [[23111879, 23593852]],
  'chr14': [[66143522, 66818672]]},
 'TSI': {'chr2': [[193268274, 193813343]], 'chr13': [[55278837, 55753073]]},
 'YRI': {'chr10': [[63150067, 63541883]],
  'chr14': [[66379055, 66778338]],
  'chr22': [[22691667, 22898645]]}}

In [13]:
def find_overlap(dict1, dict2):
    overlap_regions = {}


    common_races = set(dict1.keys()).intersection(dict2.keys())
    for race in common_races:
        overlap_regions[race] = {}

   
        common_chrs = set(dict1[race].keys()).intersection(dict2[race].keys())
        for chr in common_chrs:
            overlap_regions[race][chr] = []

   
            for region1 in dict1[race][chr]:
                for region2 in dict2[race][chr]:
          
                    if max(region1[0], region2[0]) <= min(region1[1], region2[1]):
            
                        overlap_regions[race][chr].append([max(region1[0], region2[0]), min(region1[1], region2[1])])

    return overlap_regions


overlap_regions = find_overlap(dis_split_region, ass_split_race)
# for k,v in overlap_regions.items():
#     if overlap_regions[k]!={}:

print(overlap_regions)

{'CEU': {}, 'PUR': {}, 'BEB': {}, 'JPT': {}, 'CHB': {}, 'MSL': {'chr5': []}, 'CHS': {}, 'KHV': {}, 'CLM': {}, 'PEL': {'chr11': []}, 'PJL': {}, 'ESN': {}, 'YRI': {}, 'LWK': {}, 'CDX': {}, 'ASW': {}, 'MXL': {'chr11': []}, 'IBS': {}, 'GWD': {}, 'TSI': {}, 'GBR': {}, 'STU': {'chr3': []}, 'ACB': {'chr3': [], 'chr5': []}, 'ITU': {}, 'FIN': {}, 'GIH': {'chr3': []}}


In [ ]:
mhc_repeat_region=[[32589647, 32751796]]